In [ ]:
import h5py
import numpy as np

path='/p/project/pines/li47/code/projectData/NST_b-discNJN/cD211.054.96/data_merge/NST_b.h5_cD96'
outfile='dat/NST_b.h5_cD96_2pt'
with h5py.File(path) as f, h5py.File(outfile,'w') as fw:
    fw.copy(f['cfgs'],fw,name='cfgs')
    fw.copy(f['diags/N'],fw,name='diags/N')
    print(f['diags'].keys())